##### Copyright 2021 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# nqueens_sat

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/master/examples/notebook/sat/nqueens_sat.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/master/ortools/sat/samples/nqueens_sat.py"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
!pip install ortools

In [ ]:
#!/usr/bin/env python3
# Copyright 2010-2021 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# [START program]
"""OR-Tools solution to the N-queens problem."""
# [START import]
import sys
import time
from ortools.sat.python import cp_model
# [END import]


# [START solution_printer]
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print('Solution %i, time = %f s' %
              (self.__solution_count, current_time - self.__start_time))
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i.
                    print('Q', end=' ')
                else:
                    print('_', end=' ')
            print()
        print()

# [END solution_printer]


# Creates the solver.
# [START model]
model = cp_model.CpModel()
# [END model]

# Creates the variables.
# [START variables]
# The array index is the column, and the value is the row.
queens = [
    model.NewIntVar(0, board_size - 1, 'x%i' % i) for i in range(board_size)
]
# [END variables]

# Creates the constraints.
# [START constraints]
# All rows must be different.
model.AddAllDifferent(queens)

# All columns must be different because the indices of queens are all
# different.

# No two queens can be on the same diagonal.
model.AddAllDifferent([queens[i] + i for i in range(board_size)])
model.AddAllDifferent([queens[i] - i for i in range(board_size)])
# [END constraints]

# Solve the model.
# [START solve]
solver = cp_model.CpSolver()
solution_printer = NQueenSolutionPrinter(queens)
solver.parameters.enumerate_all_solutions = True
solver.Solve(model, solution_printer)
# [END solve]

# Statistics.
# [START statistics]
print('\nStatistics')
print(f'  conflicts      : {solver.NumConflicts()}')
print(f'  branches       : {solver.NumBranches()}')
print(f'  wall time      : {solver.WallTime()} s')
print(f'  solutions found: {solution_printer.solution_count()}')
# [END statistics]

